In [1]:
import urllib
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import string as st
import seaborn as sns

In [ ]:
def top_2000_companies():
    
    r = urllib.request.urlopen('https://h1bdata.info/topcompanies.php') 
    soup = BeautifulSoup(r)

    data2 = soup.find_all('tr')    
    labels = []
    #get headings
    for h in data2[0].find_all('th'):
        labels.append(h.get_text().strip().lower())   
    final = []
    #get data
    for data in data2[1:]:
        data_list = []
        for d in data.find_all('td'):
            d_str = d.get_text().replace(',','')
            
            if d_str.isnumeric():
                data_list.append(int(d_str))
            else:
                data_list.append(d_str)                      
        final.append(data_list)
    
    df = pd.DataFrame(final, columns = labels) 
    df =df.drop(columns='latest filings')
    df = df.rename(columns={"#": "index", "company name": "company","# of h-1b filings": "total_filings","average salary":"avg_salary"})
    df = df.set_index('index')
    return df

In [ ]:
df_companies = top_2000_companies()
df_companies.to_csv('data/top2kcompanynames.csv')
df_companies = df_companies.dropna(how='any')
df_companies

In [27]:
def load_data(company,year):
    company = company.replace(' ','+')
    #print('https://h1bdata.info/index.php?em='+company+'&job=&city=&year='+str(year))
    print(company)
    r = urllib.request.urlopen('https://h1bdata.info/index.php?em='+company+'&job=&city=&year='+str(year)) 
    soup = BeautifulSoup(r)

#     data2 = soup.find_all('tr')    
#     labels = []
#     for h in data2[0].find_all('th'):
#         labels.append(h.get_text().strip().lower())

    final = []
    for data in data2[1:]:
        data_list = []
        for d in data.find_all('td'):
            d_str = d.get_text().replace(',','')

            if d_str.isnumeric():
                data_list.append(int(d_str))
            else:
                data_list.append(d_str)                      
        final.append(data_list)
    
    df = pd.DataFrame(final, columns = labels)    
    return df

In [ ]:
frames = [load_data(company,year) for company in df_companies['company'].dropna().to_list()[:10] for year in range(2012,2022) ]  
result = pd.concat(frames)

In [ ]:
result = result.rename(columns=['employer','job_title','base_salary','location','submit_date','start_date'])
result['submit_date'] = pd.to_datetime(result['submit_date'])
result['start_date'] = pd.to_datetime(result['start_date'])
#result['state'] = result['location'].str.split().str[-1] 
#result['year'] = result['submit_date'].dt.year

In [38]:
result.to_csv("top10.csv")

In [39]:
result

,employer,job title,base salary,location,submit date,start date,submit_date,start_date,year
0,INFOSYS LIMITED,ACCOUNT MANAGER,108913.0,MILWAUKEE WI,12/03/2014,04/22/2015,2014-12-03,2015-04-22,2014.0
1,INFOSYS LIMITED,ACCOUNT MANAGER,122500.0,SAINT LOUIS MO,11/04/2014,11/15/2014,2014-11-04,2014-11-15,2014.0
2,INFOSYS LIMITED,ACCOUNT MANAGER,124067.0,PHOENIX AZ,10/17/2014,10/28/2014,2014-10-17,2014-10-28,2014.0
3,INFOSYS LIMITED,ACCOUNT MANAGER,139804.0,HOUSTON TX,10/09/2014,10/20/2014,2014-10-09,2014-10-20,2014.0
4,INFOSYS LIMITED,ACCOUNT MANAGER,145933.0,FOSTER CITY CA,12/29/2014,01/09/2015,2014-12-29,2015-01-09,2014.0
...,...,...,...,...,...,...,...,...,...
34577,IBM INDIA PRIVATE LIMITED,WORKDAY LEAD CONSULTANT,83101.0,SANTA CLARA CA,12/30/2014,06/30/2015,2014-12-30,2015-06-30,2014.0
34578,IBM INDIA PRIVATE LIMITED,WORKDAY TECHNO FUNCTIONAL CONSULTANT,93642.0,NEW BRUNSWICK NJ,08/29/2018,01/01/2019,2018-08-29,2019-01-01,2018.0
34579,IBM INDIA PRIVATE LIMITED,WORKDAY TECHNO- FUNCTIONAL CONSULTANT,86044.0,NEW BRUNSWICK NJ,02/17/2017,08/07/2017,2017-02-17,2017-08-07,2017.0
34580,IBM INDIA PRIVATE LIMITED,WORKDAY TECHNO-FUNCTIONAL CONSULTANT,84864.0,NEW BRUNSWICK NJ,02/14/2017,08/07/2017,2017-02-14,2017-08-07,2017.0


In [ ]:
result.groupby('year','job title').count().reset_index()

In [ ]:
sns.scatterplot(data= result.groupby('year').count().reset_index(),x='year',y='employer')

In [ ]:
df['employer'].unique()